In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('projCasaTardeli').getOrCreate() 
dataCasaTardeli = "/FileStore/tables/CasaTardeli_04_10_23.csv"

data = spark.read.csv(dataCasaTardeli, sep=";", header=True, inferSchema=True)

# Exibir o DataFrame do Spark
data.show()

+--------------------+--------------------+------+-----+-----------+------+-----+--------+---------------+-----------+
|             Cliente|             Produto|Código| Qtde|Preço Médio| Valor|Frete|Desconto|Outras despesas|Total Venda|
+--------------------+--------------------+------+-----+-----------+------+-----+--------+---------------+-----------+
|Adailton e Willian  |                NULL|  NULL| 1,00|      67,00| 67,00| 0,00|    0,00|           0,00|      67,00|
|                    |Vinho Adega Grand...|  NULL| 1,00|      67,00| 67,00| 0,00|    0,00|           0,00|      67,00|
|            ADEPOM  |                NULL|  NULL|18,00|      43,56|784,00| 0,00|    0,00|           0,00|     784,00|
|                    |Cachaça Engenho C...|  NULL| 2,00|      65,00|130,00| 0,00|    0,00|           0,00|     130,00|
|                    |Cerveja Langerwis...|  NULL| 1,00|      26,00| 26,00| 0,00|    0,00|           0,00|      26,00|
|                    |Cerveja Langerwis...|  NUL

In [0]:
%fs ls /FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/CasaTardeli_04_10_23.csv,CasaTardeli_04_10_23.csv,66486,1697073236000
dbfs:/FileStore/tables/CasaTardeli_Transf.csv/,CasaTardeli_Transf.csv/,0,0
dbfs:/FileStore/tables/final_output.csv/,final_output.csv/,0,0


In [0]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('projCasaTardeli').getOrCreate()

df = spark.read.csv(dataCasaTardeli, sep=";", header=True, inferSchema=True)

# Converter DataFrame do Spark para um DataFrame do Pandas
df = df.toPandas()

# Setar cliente como nulo se não tiver valor definido
df['Cliente'] = df['Cliente'].str.strip().replace('', pd.NA)

# Completar linhas nulas da coluna "Cliente" com o último valor não nulo
df['Cliente'].fillna(method='ffill', inplace=True)

# Remover última linha
df = df.iloc[:-1]

# Remover linhas onde o produto está nulo
df = df.dropna(subset=['Produto'])

# Converter números em string para decimal
columns_to_convert = ['Qtde', 'Preço Médio', 'Valor', 'Frete', 'Desconto', 'Outras despesas', 'Total Venda']
df[columns_to_convert] = df[columns_to_convert].applymap(lambda x: float(x.replace('.', '').replace(',', '.')))
df = df.drop("Código", axis=1)

# Exibir o DataFrame do Pandas
#print(df)

# Se você precisar converter o DataFrame do Pandas de volta para um DataFrame do Spark, você pode fazer isso:
data = spark.createDataFrame(df)
data.show()




+------------------+--------------------+----+-----------+-----+-----+--------+---------------+-----------+
|           Cliente|             Produto|Qtde|Preço Médio|Valor|Frete|Desconto|Outras despesas|Total Venda|
+------------------+--------------------+----+-----------+-----+-----+--------+---------------+-----------+
|Adailton e Willian|Vinho Adega Grand...| 1.0|       67.0| 67.0|  0.0|     0.0|            0.0|       67.0|
|            ADEPOM|Cachaça Engenho C...| 2.0|       65.0|130.0|  0.0|     0.0|            0.0|      130.0|
|            ADEPOM|Cerveja Langerwis...| 1.0|       26.0| 26.0|  0.0|     0.0|            0.0|       26.0|
|            ADEPOM|Cerveja Langerwis...| 4.0|       25.0|100.0|  0.0|     0.0|            0.0|      100.0|
|            ADEPOM|Cerveja Langerwis...| 1.0|       29.0| 29.0|  0.0|     0.0|            0.0|       29.0|
|            ADEPOM|Cerveja Langerwis...| 2.0|       15.0| 30.0|  0.0|     0.0|            0.0|       30.0|
|            ADEPOM|      Pr

In [0]:
# Salvando versão final transformada em ".cvs" patra manusear no DataBricks, Power Query, etc.
df = data.toPandas()  # Convertendo para um DataFrame do Pandas
df.to_csv("enoteca.csv", index=True)  # Salvando como CSV
